# Using `sklearn` Surrogates in `spotPython` {#sec-sklearn-surrogates}

Besides the internal kriging surrogate, which is used as a default by `spotPython`, any surrogate model from `scikit-learn` can be used as a surrogate in `spotPython`. This chapter explains how to use `scikit-learn` surrogates in `spotPython`.


In [1]:
import numpy as np
from math import inf
from spotPython.fun.objectivefunctions import analytical
from spotPython.spot import spot

Seed set to 123


## Example: Branin Function with `spotPython`'s Internal Kriging Surrogate

### The Objective Function Branin

* The `spotPython` package provides several classes of objective functions.
* We will use an analytical objective function, i.e., a function that can be described by a (closed) formula.
* Here we will use the Branin function:

        y = a * (x2 - b * x1**2 + c * x1 - r) ** 2 + s * (1 - t) * np.cos(x1) + s,
        where values of a, b, c, r, s and t are: a = 1, b = 5.1 / (4*pi**2),
        c = 5 / pi, r = 6, s = 10 and t = 1 / (8*pi).

* It has three global minima:


        f(x) = 0.397887 at (-pi, 12.275), (pi, 2.275), and (9.42478, 2.475).


In [2]:
from spotPython.fun.objectivefunctions import analytical
fun = analytical().fun_branin

:::{.callout-note}
#### TensorBoard

Similar to the one-dimensional case, which was introduced in Section @sec-visualizing-tensorboard-01, we can use TensorBoard to monitor the progress of the optimization. We will use the same code, only the prefix is different:


In [3]:
from spotPython.utils.init import fun_control_init, design_control_init
PREFIX = "04"
fun_control = fun_control_init(
    PREFIX=PREFIX,
    lower = np.array([-5,-0]),
    upper = np.array([10,15]),
    fun_evals=20,
    max_time=inf)

design_control = design_control_init(
    init_size=10)

Seed set to 123


Created spot_tensorboard_path: runs/spot_logs/04_maans14_2024-01-09_07-58-38 for SummaryWriter()


:::

### Running the surrogate model based optimizer `Spot`:


In [4]:
spot_2 = spot.Spot(fun=fun,
                   fun_control=fun_control,
                   design_control=design_control)

In [5]:
spot_2.run()

spotPython tuning: 3.1468324340130396 [######----] 55.00% 


spotPython tuning: 3.1468324340130396 [######----] 60.00% 


spotPython tuning: 3.1468324340130396 [######----] 65.00% 


spotPython tuning: 3.1468324340130396 [#######---] 70.00% 


spotPython tuning: 1.1487178651597567 [########--] 75.00% 


spotPython tuning: 1.023873960670663 [########--] 80.00% 


spotPython tuning: 0.4157039136851921 [########--] 85.00% 


spotPython tuning: 0.40177808730611275 [#########-] 90.00% 


spotPython tuning: 0.39909168407470474 [##########] 95.00% 


spotPython tuning: 0.39909168407470474 [##########] 100.00% Done...



### TensorBoard

Now we can start TensorBoard in the background with the following command:



```{raw}
tensorboard --logdir="./runs"
```



We can access the TensorBoard web server with the following URL:



```{raw}
http://localhost:6006/
```



The TensorBoard plot illustrates how `spotPython` can be used as a microscope for the internal mechanisms of the surrogate-based optimization process. Here, one important parameter, the learning rate $\theta$ of the Kriging surrogate is plotted against the number of optimization steps.

![TensorBoard visualization of the spotPython optimization process and the surrogate model.](figures_static/04_tensorboard_01.png){width="100%"}



### Print the Results


In [6]:
spot_2.print_results()

min y: 0.39909168407470474
x0: 3.1523096108334228
x1: 2.292209322210569


[['x0', 3.1523096108334228], ['x1', 2.292209322210569]]

### Show the Progress and the Surrogate


In [7]:
spot_2.plot_progress(log_y=True)

<Figure size 2700x1800 with 1 Axes>

In [8]:
spot_2.surrogate.plot()

<Figure size 2700x1800 with 6 Axes>

## Example: Using Surrogates From scikit-learn

* Default is the `spotPython` (i.e., the internal) `kriging` surrogate.
* It can be called explicitely and passed to `Spot`.


In [9]:
from spotPython.build.kriging import Kriging
S_0 = Kriging(name='kriging', seed=123)

* Alternatively, models from `scikit-learn` can be selected, e.g., Gaussian Process, RBFs, Regression Trees, etc.


In [10]:
# Needed for the sklearn surrogates:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import tree
import pandas as pd

* Here are some additional models that might be useful later:


In [11]:
S_Tree = DecisionTreeRegressor(random_state=0)
S_LM = linear_model.LinearRegression()
S_Ridge = linear_model.Ridge()
S_RF = RandomForestRegressor(max_depth=2, random_state=0)

### GaussianProcessRegressor as a Surrogate

* To use a Gaussian Process model from `sklearn`, that is similar to `spotPython`'s `Kriging`, we can proceed as follows:


In [12]:
kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
S_GP = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)

* The scikit-learn GP model `S_GP` is selected for `Spot` as follows: 

    `surrogate = S_GP`

* We can check the kind of surogate model with the command `isinstance`:


In [13]:
isinstance(S_GP, GaussianProcessRegressor) 

True

In [14]:
isinstance(S_0, Kriging)

True

* Similar to the `Spot` run with the internal `Kriging` model, we can call the run with the `scikit-learn` surrogate:


In [15]:
fun = analytical(seed=123).fun_branin
spot_2_GP = spot.Spot(fun=fun,
                     fun_control=fun_control,
                     design_control=design_control,
                     surrogate = S_GP)
spot_2_GP.run()

spotPython tuning: 18.865121449825782 [######----] 55.00% 


spotPython tuning: 4.06700305855078 [######----] 60.00% 


spotPython tuning: 3.461906927549384 [######----] 65.00% 


spotPython tuning: 3.461906927549384 [#######---] 70.00% 


spotPython tuning: 1.3280944252046556 [########--] 75.00% 


...

spotPython tuning: 0.39916716809341857 [#########-] 90.00% 


/Users/bartz/miniforge3/envs/spotCondaEnv/lib/python3.11/site-packages/sklearn/gaussian_process/_gpr.py:663: ConvergenceWarning:

lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html

/Users/bartz/miniforge3/envs/spotCondaEnv/lib/python3.11/site-packages/sklearn/gaussian_process/_gpr.py:663: ConvergenceWarning:

lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



spotPython tuning: 0.3982254000779708 [##########] 95.00% 


spotPython tuning: 0.3982254000779708 [##########] 100.00% Done...



/Users/bartz/miniforge3/envs/spotCondaEnv/lib/python3.11/site-packages/sklearn/gaussian_process/_gpr.py:663: ConvergenceWarning:

lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html

/Users/bartz/miniforge3/envs/spotCondaEnv/lib/python3.11/site-packages/sklearn/gaussian_process/_gpr.py:663: ConvergenceWarning:

lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



In [16]:
spot_2_GP.plot_progress()

<Figure size 2700x1800 with 1 Axes>

In [17]:
spot_2_GP.print_results()

min y: 0.3982254000779708
x0: 3.1499822680266343
x1: 2.268811272474469


[['x0', 3.1499822680266343], ['x1', 2.268811272474469]]

## Example: One-dimensional Sphere Function With `spotPython`'s Kriging

* In this example, we will use an one-dimensional function, which allows us to visualize the optimization process.
  * `show_models= True` is added to the argument list.


In [18]:
from spotPython.fun.objectivefunctions import analytical
fun_control = fun_control_init(
    lower = np.array([-1]),
    upper = np.array([1]),
    fun_evals=10,
    max_time=inf,
    show_models= True,
    tolerance_x = np.sqrt(np.spacing(1)))
fun = analytical(seed=123).fun_sphere
design_control = design_control_init(
    init_size=3)

Seed set to 123


In [19]:
spot_1 = spot.Spot(fun=fun,
                    fun_control=fun_control,
                    design_control=design_control)
spot_1.run()

<Figure size 1650x1050 with 1 Axes>

<Figure size 1650x1050 with 1 Axes>

spotPython tuning: 0.03475493366922229 [####------] 40.00% 


<Figure size 1650x1050 with 1 Axes>

spotPython tuning: 0.03475493366922229 [#####-----] 50.00% 


...

spotPython tuning: 4.643556417950029e-08 [########--] 80.00% 


<Figure size 1650x1050 with 1 Axes>

spotPython tuning: 4.643556417950029e-08 [#########-] 90.00% 


<Figure size 1650x1050 with 1 Axes>

spotPython tuning: 4.643556417950029e-08 [##########] 100.00% Done...



### Results


In [20]:
spot_1.print_results()

min y: 4.643556417950029e-08
x0: -0.0002154891277524235


[['x0', -0.0002154891277524235]]

In [21]:
spot_1.plot_progress(log_y=True)

<Figure size 2700x1800 with 1 Axes>

* The method `plot_model` plots the final surrogate:


In [22]:
spot_1.plot_model()

<Figure size 1650x1050 with 1 Axes>

## Example: `Sklearn` Model GaussianProcess

* This example visualizes the search process on the `GaussianProcessRegression` surrogate from `sklearn`.
* Therefore `surrogate = S_GP` is added to the argument list.


In [23]:
fun = analytical(seed=123).fun_sphere
spot_1_GP = spot.Spot(fun=fun,
                      fun_control=fun_control,
                      design_control=design_control,
                      surrogate = S_GP)
spot_1_GP.run()

<Figure size 1650x1050 with 1 Axes>

<Figure size 1650x1050 with 1 Axes>

spotPython tuning: 0.004925671374769521 [####------] 40.00% 


/Users/bartz/miniforge3/envs/spotCondaEnv/lib/python3.11/site-packages/sklearn/gaussian_process/_gpr.py:663: ConvergenceWarning:

lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



<Figure size 1650x1050 with 1 Axes>

...

<Figure size 1650x1050 with 1 Axes>

spotPython tuning: 2.2493573831304313e-10 [#########-] 90.00% 


/Users/bartz/miniforge3/envs/spotCondaEnv/lib/python3.11/site-packages/sklearn/gaussian_process/_gpr.py:663: ConvergenceWarning:

lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



<Figure size 1650x1050 with 1 Axes>

spotPython tuning: 2.2493573831304313e-10 [##########] 100.00% Done...



In [24]:
spot_1_GP.print_results()

min y: 2.2493573831304313e-10
x0: 1.499785779079943e-05


[['x0', 1.499785779079943e-05]]

In [25]:
spot_1_GP.plot_progress(log_y=True)

<Figure size 2700x1800 with 1 Axes>

In [26]:
spot_1_GP.plot_model()

<Figure size 1650x1050 with 1 Axes>

## Exercises

### `DecisionTreeRegressor`

* Describe the surrogate model.
* Use the surrogate as the model for optimization.

### `RandomForestRegressor`

* Describe the surrogate model.
* Use the surrogate as the model for optimization.

### `linear_model.LinearRegression` 

* Describe the surrogate model.
* Use the surrogate as the model for optimization.

### `linear_model.Ridge`

* Describe the surrogate model.
* Use the surrogate as the model for optimization.

## Exercise 2

* Compare the performance of the five different surrogates on both objective functions:
  
  * `spotPython`'s internal Kriging
  * `DecisionTreeRegressor`
  * `RandomForestRegressor`
  * `linear_model.LinearRegression`
  * `linear_model.Ridge`
